## Fast loops using just in time compilation

In [ ]:
# import the necessary packages
import matplotlib.pyplot as plt
import cv2

from uk_utils import imshow

In [ ]:
# load the original image, convert it to grayscale, and display
# it inline
img_name = '../../images/seeds.tif'
image = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)

if image is None:
    print('Image ', img_name, ' not found')
else:
    imshow(image)

In [ ]:
image.shape

## Cython

In [ ]:
# If you don't have installed cython you can install it from within the notebook
#!pip install cython

In [ ]:
%load_ext Cython

In [ ]:
def threshold_slow(T, image):
    # grab the image dimensions
    M = image.shape[0]
    N = image.shape[1]
    
    # loop over the image, pixel by pixel
    for y in range(0, M):
        for x in range(0, N):
            # threshold the pixel
            image[y, x] = 255 if image[y, x] >= T else 0
            
    # return the thresholded image
    return image

In [ ]:
img2 = threshold_slow(180, image)
imshow(img2)

In [ ]:
%timeit threshold_slow(5, image)

In [ ]:
%%cython -a
import cython

@cython.boundscheck(False)
cpdef unsigned char[:, :] threshold_fast(int T, unsigned char [:, :] image):
    # set the variable extension types
    cdef int x, y, N, M
    
    # grab the image dimensions
    M = image.shape[0]
    N = image.shape[1]
    
    # loop over the image
    for y in range(0, M):
        for x in range(0, N):
            # threshold the pixel
            image[y, x] = 255 if image[y, x] >= T else 0
    
    # return the thresholded image
    return image

In [ ]:
# reload the original image and convert it to grayscale
image = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)

In [ ]:
%timeit threshold_fast(5, image)

## Numba

In [ ]:
from numba import jit

In [ ]:
@jit(parallel=False)
def threshold(T, image):
    # grab the image dimensions
    M = image.shape[0]
    N = image.shape[1]
    
    # loop over the image, pixel by pixel
    for y in range(0, M):
        for x in range(0, N):
            # threshold the pixel
            image[y, x] = 255 if image[y, x] >= T else 0
            
    # return the thresholded image
    return image

In [ ]:
%timeit threshold(5, image)